# Training

In questo script viene effettuato il training dei dati a secondo del dataset scelto e visualizzati i risultati ottenuti con una matrice di confusione e una tabella di Metric Accuracy

## Import di liberie

In [2]:
import numpy as np
import dlib
import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
from sklearn.utils import resample
from sklearn import metrics
import dlib.cuda as cuda

Using TensorFlow backend.
/home/francesco/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/francesco/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/francesco/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/francesco/.local/lib/python3.5/site-packages/tenso

## Caricamento di train e test set per ogni dataset

In [3]:
#Caricamento di train e test set del dataset1 - CelebA
X1_train = np.load('dataset1/X1_train.npy')
X1_test = np.load('dataset1/X1_test.npy')
y1_train = np.load('dataset1/y1_train.npy')
y1_test = np.load('dataset1/y1_test.npy')

#Caricamento di train e test set del dataset2 - WheDataset
X2_train = np.load('dataset2/X2_train.npy')
X2_test = np.load('dataset2/X2_test.npy')
y2_train = np.load('dataset2/y2_train.npy')
y2_test = np.load('dataset2/y2_test.npy')

#Caricamento di train e test set del dataset3 - CelebA + WheDataset
X3_train = np.load('dataset3/X3_train.npy')
X3_test = np.load('dataset3/X3_test.npy')
y3_train = np.load('dataset3/y3_train.npy')
y3_test = np.load('dataset3/y3_test.npy')

## Rete Neurale (ANN)

In questa sezione sono presenti le due reti neurali utilizzate per lo sviluppo di questo progetto. La CNN1 è stata la prima rete neurale che è stata sperimentata ma non ha raggiunto ottime performance sul test set. La seconda rete CNN2 invece ha raggiunto risultati migliori.

In [ ]:
classifier = Sequential()
'''
#ANN1
classifier.add(Dense(output_dim = 128,  activation = 'relu', input_dim = 64))
classifier.add(Dense(output_dim = 64, activation = 'relu'))
classifier.add(Dense(output_dim = 32, activation = 'relu'))
classifier.add(Dense(output_dim = 16, activation = 'relu'))
classifier.add(Dense(output_dim = 8, activation = 'relu'))
classifier.add(Dense(output_dim = 1,  activation = 'sigmoid')) # sigmoid 
'''

#ANN2
classifier.add(Dense(output_dim = 64,  activation = 'relu', input_dim = 64))
classifier.add(Dropout(0.2)) #ridimensiona i pesi del 20% per ridurre l'overfitting
classifier.add(Dense(output_dim = 32, activation = 'relu'))
classifier.add(Dense(output_dim = 32, activation = 'relu'))
classifier.add(Dropout(0.2)) #ridimensiona i pesi del 20% per ridurre l'overfitting
classifier.add(Dense(output_dim = 16, activation = 'relu'))
classifier.add(Dense(output_dim = 8, activation = 'relu'))
classifier.add(Dense(output_dim = 1,  activation = 'sigmoid')) # sigmoid 

In questa sezione è presenbte la compliazione e l'addestramento dei dati.

In [ ]:
tensorboard = TensorBoard(log_dir = "logs/{}".format(time())) #salva i grafici di accuracy nella cartella logs


classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy' , metrics = ['accuracy'] ) 


epoche = 100
batch_size = 32

#Settaggio con dataset 1 - CelebA
classifier.fit(x=X1_train, y=y1_train, batch_size=batch_size, epochs=epoche, callbacks=[tensorboard])

Questa parte salva i pesi della ANN, in modo da poterla riutilizzarla nuovamente senza effettuare nuovamente l'addestramento.

In [ ]:
#Settaggio per ANN2
stringNameANN = 'ANN2 - '+'ep:'+str(epoche)+' - bs:'+str(batch_size)
classifier.save(stringNameANN)

## Risultati ottenuti

In questa ultima sezione vengono mostrati i risultati ottenuti con una matrice di confusione e una tabella di Metric Accuracy.

In [ ]:
#Settaggio per dataset 1 - CelebA
y_pred= classifier.predict(X1_test)

y_pred_bool = (y_pred>0.5)

print('Confusion Matrix')
print(confusion_matrix(y1_test,y_pred_bool))

print('Metric Accuracy')
print(metrics.classification_report(y1_test,y_pred_bool))